# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from sa_models.SA_BERT_Dropout_BiLSTM_Linear import SA_BERT_Dropout_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [7]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [8]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [9]:
BERT_FINE_TUNED_PATH = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [10]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_bilstm_linear.csv'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
best_accuracy = 0.0

In [13]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [14]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [15]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [16]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = SA_BERT_Dropout_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6645181775093079
Epoch: 0/2, Batch: 31/313, Loss: 0.5710276365280151
Epoch: 0/2, Batch: 62/313, Loss: 0.5204684138298035
Epoch: 0/2, Batch: 93/313, Loss: 0.4524083137512207
Epoch: 0/2, Batch: 124/313, Loss: 0.40011662244796753
Epoch: 0/2, Batch: 155/313, Loss: 0.3519105911254883
Epoch: 0/2, Batch: 186/313, Loss: 0.28655576705932617
Epoch: 0/2, Batch: 217/313, Loss: 0.2627926468849182
Epoch: 0/2, Batch: 248/313, Loss: 0.4218378961086273
Epoch: 0/2, Batch: 279/313, Loss: 0.20116674900054932
Epoch: 0/2, Batch: 310/313, Loss: 0.19783291220664978
Epoch: 1/2, Batch: 0/313, Loss: 0.19894421100616455
Epoch: 1/2, Batch: 31/313, Loss: 0.1574559509754181
Epoch: 1/2, Batch: 62/313, Loss: 0.1697314977645874
Epoch: 1/2, Batch: 93/313, Loss: 0.15206924080848694
Epoch: 1/2, Batch: 124/313, Loss: 0.1686207354068756
Epoch: 1/2, Batch: 155/313, Loss: 0.1941649615764618
Epoch: 1/2, Batch: 186/313, Loss: 0.12248961627483368
Epoch: 1/2, Batch: 217/313, Loss: 0.1167

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6979970932006836
Epoch: 0/2, Batch: 31/313, Loss: 0.6100785136222839
Epoch: 0/2, Batch: 62/313, Loss: 0.5069830417633057
Epoch: 0/2, Batch: 93/313, Loss: 0.42375585436820984
Epoch: 0/2, Batch: 124/313, Loss: 0.40684303641319275
Epoch: 0/2, Batch: 155/313, Loss: 0.3803372383117676
Epoch: 0/2, Batch: 186/313, Loss: 0.2937946617603302
Epoch: 0/2, Batch: 217/313, Loss: 0.47622525691986084
Epoch: 0/2, Batch: 248/313, Loss: 0.2602068781852722
Epoch: 0/2, Batch: 279/313, Loss: 0.2639661133289337
Epoch: 0/2, Batch: 310/313, Loss: 0.19133317470550537
Epoch: 1/2, Batch: 0/313, Loss: 0.18786299228668213
Epoch: 1/2, Batch: 31/313, Loss: 0.19723889231681824
Epoch: 1/2, Batch: 62/313, Loss: 0.2469259798526764
Epoch: 1/2, Batch: 93/313, Loss: 0.16662488877773285
Epoch: 1/2, Batch: 124/313, Loss: 0.5002351999282837
Epoch: 1/2, Batch: 155/313, Loss: 0.19112686812877655
Epoch: 1/2, Batch: 186/313, Loss: 0.18724791705608368
Epoch: 1/2, Batch: 217/313, Loss: 0.15

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6684805154800415
Epoch: 0/2, Batch: 31/313, Loss: 0.6185739040374756
Epoch: 0/2, Batch: 62/313, Loss: 0.5731126070022583
Epoch: 0/2, Batch: 93/313, Loss: 0.4253632426261902
Epoch: 0/2, Batch: 124/313, Loss: 0.3914795517921448
Epoch: 0/2, Batch: 155/313, Loss: 0.3622531592845917
Epoch: 0/2, Batch: 186/313, Loss: 0.28150075674057007
Epoch: 0/2, Batch: 217/313, Loss: 0.2828431725502014
Epoch: 0/2, Batch: 248/313, Loss: 0.24874542653560638
Epoch: 0/2, Batch: 279/313, Loss: 0.4414605498313904
Epoch: 0/2, Batch: 310/313, Loss: 0.25127458572387695
Epoch: 1/2, Batch: 0/313, Loss: 0.18581831455230713
Epoch: 1/2, Batch: 31/313, Loss: 0.1976439356803894
Epoch: 1/2, Batch: 62/313, Loss: 0.19377169013023376
Epoch: 1/2, Batch: 93/313, Loss: 0.1932772845029831
Epoch: 1/2, Batch: 124/313, Loss: 0.3772749900817871
Epoch: 1/2, Batch: 155/313, Loss: 0.16585761308670044
Epoch: 1/2, Batch: 186/313, Loss: 0.11506471037864685
Epoch: 1/2, Batch: 217/313, Loss: 0.1233

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7076452970504761
Epoch: 0/2, Batch: 31/313, Loss: 0.5814493894577026
Epoch: 0/2, Batch: 62/313, Loss: 0.5450956225395203
Epoch: 0/2, Batch: 93/313, Loss: 0.49474066495895386
Epoch: 0/2, Batch: 124/313, Loss: 0.392852246761322
Epoch: 0/2, Batch: 155/313, Loss: 0.35905858874320984
Epoch: 0/2, Batch: 186/313, Loss: 0.33445727825164795
Epoch: 0/2, Batch: 217/313, Loss: 0.31031662225723267
Epoch: 0/2, Batch: 248/313, Loss: 0.2723121643066406
Epoch: 0/2, Batch: 279/313, Loss: 0.4417456090450287
Epoch: 0/2, Batch: 310/313, Loss: 0.20565387606620789
Epoch: 1/2, Batch: 0/313, Loss: 0.21847018599510193
Epoch: 1/2, Batch: 31/313, Loss: 0.4763028621673584
Epoch: 1/2, Batch: 62/313, Loss: 0.1858019232749939
Epoch: 1/2, Batch: 93/313, Loss: 0.17631709575653076
Epoch: 1/2, Batch: 124/313, Loss: 0.1787908524274826
Epoch: 1/2, Batch: 155/313, Loss: 0.1506909430027008
Epoch: 1/2, Batch: 186/313, Loss: 0.1124819666147232
Epoch: 1/2, Batch: 217/313, Loss: 0.16511

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6880056262016296
Epoch: 0/2, Batch: 31/313, Loss: 0.6034069061279297
Epoch: 0/2, Batch: 62/313, Loss: 0.5228140354156494
Epoch: 0/2, Batch: 93/313, Loss: 0.45255351066589355
Epoch: 0/2, Batch: 124/313, Loss: 0.44525179266929626
Epoch: 0/2, Batch: 155/313, Loss: 0.3868181109428406
Epoch: 0/2, Batch: 186/313, Loss: 0.325389564037323
Epoch: 0/2, Batch: 217/313, Loss: 0.33383506536483765
Epoch: 0/2, Batch: 248/313, Loss: 0.230413556098938
Epoch: 0/2, Batch: 279/313, Loss: 0.3503146767616272
Epoch: 0/2, Batch: 310/313, Loss: 0.4904465675354004
Epoch: 1/2, Batch: 0/313, Loss: 0.24769684672355652
Epoch: 1/2, Batch: 31/313, Loss: 0.19939641654491425
Epoch: 1/2, Batch: 62/313, Loss: 0.17969956994056702
Epoch: 1/2, Batch: 93/313, Loss: 0.20483747124671936
Epoch: 1/2, Batch: 124/313, Loss: 0.13021335005760193
Epoch: 1/2, Batch: 155/313, Loss: 0.4762203097343445
Epoch: 1/2, Batch: 186/313, Loss: 0.14796553552150726
Epoch: 1/2, Batch: 217/313, Loss: 0.445

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.929487,0.929487,0.600377,0.929487,0.650305,0.929487,0.622712,207.692270
1,0.942308,0.942308,0.620755,0.942308,0.649246,0.942308,0.634637,193.893835
2,0.955128,0.955128,0.627701,0.955128,0.652426,0.955128,0.639722,192.380167
3,0.942308,0.942308,0.629952,0.942308,0.642424,0.942308,0.635782,192.418129
4,0.945513,0.945513,0.619564,0.945513,0.646128,0.945513,0.632275,192.523239
5,0.955128,0.955128,0.634226,0.955128,0.649947,0.955128,0.641987,192.398958
6,0.916667,0.916667,0.592511,0.916667,0.637209,0.916667,0.609835,192.352335
7,0.942308,0.942308,0.618701,0.942308,0.645123,0.942308,0.631343,192.382983
8,0.919872,0.919872,0.618186,0.919872,0.620000,0.919872,0.617710,192.377663
9,0.942308,0.942308,0.633070,0.942308,0.636054,0.942308,0.633849,192.510810


In [18]:
results.to_csv(STATS_OUTPUT)